# Word2Vec: interpretação da linguagem humana com Word embedding

Material referente aos cursos da Alura disponíveis [aqui](https://cursos.alura.com.br/course/introducao-word-embedding) e [aqui](https://cursos.alura.com.br/course/word2vec-treinamento-word-embedding). O propósito final deste notebook é criar um detector de categoria de notícia.

## Preparando o ambiente

In [1]:
!pip install gensim
!pip install -U pip setuptools wheel --user --no-warn-script-location
!pip install -U spacy
!python -m spacy download pt_core_news_sm


[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [81]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from gensim.models import KeyedVectors, Word2Vec
import nltk
import string
import numpy as np
import spacy
import pickle
import logging

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\salat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
SEED = 42

## Carregando os dados

In [5]:
treino = pd.read_csv('dados/treino.csv')
teste = pd.read_csv('dados/teste.csv')

In [6]:
treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [7]:
with open('dados/cbow_s300.txt') as f:
    for linha in range(10):
        print(next(f))

929606 300

</s> -0.001667 -0.000158 -0.000026 0.001300 -0.000796 0.001527 0.000046 0.000584 0.000449 -0.000100 0.000353 0.001251 0.001069 0.000506 0.000574 0.000838 -0.000930 -0.001220 0.000317 0.001315 -0.001120 0.001373 -0.000040 -0.001580 0.000421 -0.000667 -0.001556 -0.000746 0.001604 0.001157 -0.000027 0.000354 0.000358 -0.000527 -0.000573 -0.001512 -0.001557 -0.001637 0.001617 -0.001511 -0.001022 -0.001426 0.001086 -0.001033 0.000593 0.000724 0.000627 -0.000450 -0.001140 0.000333 0.000524 0.001541 0.000284 0.000617 -0.000807 -0.000088 -0.000364 0.001126 -0.001230 -0.001138 -0.001280 0.001330 0.001257 0.000576 0.000764 0.000684 0.001008 -0.000215 -0.000629 -0.001228 -0.001557 -0.000311 -0.000246 0.000045 0.001136 -0.000645 -0.000549 0.001099 0.000858 -0.000886 0.000553 0.000303 0.001433 0.000732 0.001321 -0.000894 -0.000700 -0.000661 -0.001484 -0.000950 -0.001556 -0.000809 0.000348 -0.000068 0.000724 -0.000569 -0.000161 -0.001628 -0.001437 -0.000259 -0.000296 -0.001571 0.000149 0

In [8]:
modelo = KeyedVectors.load_word2vec_format('dados/cbow_s300.txt')

In [9]:
modelo.get_vector('china')

array([-1.49033e-01,  1.26020e-01,  2.17628e-01,  1.82684e-01,
        1.65151e-01, -1.59660e-01, -2.34411e-01,  6.00570e-02,
        8.03680e-02,  2.87578e-01, -4.81100e-03, -5.68800e-02,
        2.15676e-01,  8.65540e-02,  1.25983e-01,  3.36157e-01,
       -1.83254e-01, -1.18499e-01,  1.13010e-02,  1.03814e-01,
        9.37640e-02,  2.90178e-01, -1.64395e-01, -1.13300e-02,
       -1.80676e-01, -1.15820e-02,  1.08728e-01,  1.65898e-01,
        9.37900e-02,  2.66767e-01, -1.29890e-02,  9.16030e-02,
        2.21292e-01, -1.36497e-01, -4.26350e-02, -1.30038e-01,
        2.17067e-01, -1.01963e-01, -3.70960e-02,  1.42155e-01,
        3.41109e-01,  2.46560e-01,  1.27458e-01,  5.72360e-02,
       -1.47962e-01, -1.60290e-02,  1.86533e-01,  7.71550e-02,
       -3.50024e-01, -4.06085e-01,  1.67131e-01, -4.75230e-02,
        5.13780e-02, -1.28224e-01,  1.06580e-02, -2.92652e-01,
        1.40540e-01, -4.57049e-01,  1.31094e-01,  2.03234e-01,
        2.94019e-01,  7.38370e-02,  1.11554e-01, -1.642

In [10]:
len(modelo.get_vector('china'))

300

## Testando a vetorização das palavras

In [11]:
texto = [
    "tenha um bom dia",
    "tenha um péssimo dia",
    "tenha um ótimo dia"
]

In [12]:
vetorizador = CountVectorizer()
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

{'tenha': 3, 'um': 4, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 5}


In [13]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0 0]]


Esse formato não permite aproximação de palavras por contextualização.

## Obtendo as palavras mais próximas de outra

No caso, utilizando a palavra _china_.

In [14]:
modelo.most_similar('china')

[('rússia', 0.7320705652236938),
 ('índia', 0.7241616249084473),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741336584091187),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.6581848859786987),
 ('urss', 0.6581668853759766),
 ('grã-bretanha', 0.6568098068237305)]

## Explorando as relações entre as palavras

### Plural

_**Nuvens** está para **nuvem** assim como **estrelas** está para **estrela**_

$Ns => N : Es => E$

In [15]:
modelo.most_similar(positive=['nuvens', 'estrela'], negative=['nuvem'])

[('estrelas', 0.5497430562973022),
 ('plêiades', 0.3791979253292084),
 ('colinas', 0.3746805489063263),
 ('trovoadas', 0.37370333075523376),
 ('sombras', 0.37341946363449097),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727698802948),
 ('cigarras', 0.36065396666526794),
 ('galáxias', 0.35754910111427307),
 ('luas', 0.3575344979763031)]

Note que a palavra mais similar é também o plural. Ou seja, operações matemáticas estão permitindo encontrar relações de contexto em um vocabulário. No entanto nem sempre isso funciona corretamente:

### Gênero

In [16]:
modelo.most_similar(positive=['médico', 'mulher'], negative=['homem'])

[('enfermeira', 0.6180862188339233),
 ('psicóloga', 0.47447532415390015),
 ('dama-de-companhia', 0.47382351756095886),
 ('esposa', 0.46559131145477295),
 ('parteira', 0.4636286795139313),
 ('mãe', 0.45817679166793823),
 ('governanta', 0.45722925662994385),
 ('madrasta', 0.4569782614707947),
 ('menina', 0.4443591237068176),
 ('filha', 0.4434111416339874)]

Isso é um exemplo de _bias_. As nossas bases de dados refletem os viéses existentes na sociedade. Normalmente _médico_ é associado ao homem e, para mesma região contextual, o mais próximo de mulher é _enfermeira_ e não _médica_.

In [17]:
modelo.most_similar(positive=['professor', 'mulher'], negative=['homem'])

[('professora', 0.6192208528518677),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978230595588684),
 ('ex-aluna', 0.4884248375892639),
 ('namorada', 0.4737858474254608),
 ('enfermeira', 0.4728143811225891),
 ('filha', 0.4673738479614258),
 ('irmã', 0.45845916867256165),
 ('ex-namorada', 0.45824769139289856),
 ('ex-professora', 0.4510470926761627)]

Note como que para o caso de _professor/professora_ o resultado foi correto.

## Vetorização do texto: Alura News

In [18]:
treino.title.loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

In [19]:
def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerico = []
    for token_valido in nltk.word_tokenize(texto):
        if(token_valido in string.punctuation): continue
        lista_alfanumerico.append(token_valido)
    return lista_alfanumerico

In [20]:
tokenizador("Text exemplo, 1234.")

['text', 'exemplo', '1234']

In [21]:
def combinacao_de_vetores_por_soma(palavras_numeros):
    vetor_resultante = np.zeros(300)
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            if(pn.isnumeric()):
                pn = "0" * len(pn)
            else:
                pn = "unknown"
            vetor_resultante += modelo.get_vector(pn)
    return vetor_resultante

In [22]:
combinacao_de_vetores_por_soma(tokenizador("Texto exemplo blateste 123"))

array([ 2.82375004e-01,  5.34099996e-01, -9.02680010e-02,  7.14331999e-01,
       -3.14585987e-01, -2.63855997e-01,  9.72592972e-01,  2.44773991e-01,
       -4.04193003e-01, -2.54065996e-01,  7.71848995e-01,  2.93405998e-01,
       -3.74439992e-01,  6.02699012e-01, -8.82209949e-02,  1.79990008e-02,
       -1.40151996e+00, -6.86359927e-02, -3.07091993e-01, -5.15361995e-01,
       -6.40697980e-01, -7.17419907e-02,  4.17280020e-02,  7.58208998e-01,
       -3.10904011e-01,  2.48399377e-03, -6.77058031e-01, -2.58959904e-02,
       -2.53299996e-03, -1.92691989e-01,  9.42480136e-02, -3.40558998e-01,
        1.07177000e-01,  6.05130000e-02,  1.33777995e-01,  3.31810001e-01,
        2.55109005e-01,  7.39010982e-01,  9.77540053e-02,  1.62625000e-01,
       -1.29702008e-01,  2.73874013e-01,  2.90505003e-01,  2.74922011e-01,
        3.16785989e-01,  2.42439991e-01,  4.11559002e-01,  5.24873011e-01,
        1.74969999e-01,  4.76715006e-01,  7.85149941e-02,  6.02812994e-01,
        3.25109884e-02,  

### Aplicando o tokenizador para treino e teste

In [23]:
def matriz_vetores(textos):
    x = len(textos)
    matriz = np.zeros((x, 300))
    for i in range(x):
        tokens = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(tokens)
    return matriz

In [24]:
matriz_treino = matriz_vetores(treino.title)
matriz_teste = matriz_vetores(teste.title)

In [25]:
matriz_treino

array([[ 0.52357099,  1.00529201,  0.26381801, ...,  0.43726099,
         0.187696  ,  0.15216099],
       [ 0.34492697,  0.83511301, -0.88874401, ...,  0.19878698,
        -1.40402502,  1.49788603],
       [ 0.050757  ,  0.95369402,  0.63757797, ...,  0.252318  ,
         0.142377  , -1.36338596],
       ...,
       [-0.89244701,  0.71595099,  0.332483  , ...,  0.40761799,
         0.558855  , -0.04226601],
       [-0.03248802, -0.259999  , -0.925543  , ..., -0.34076802,
        -1.12714999,  0.55980401],
       [ 0.46499701,  1.07129601, -0.34205199, ..., -0.83202901,
        -0.27768998, -0.34096499]])

## Aplicando a regressão logística nos vetores encontrados

In [26]:
#LR = LogisticRegression(solver='saga', max_iter=100, multi_class='multinomial')
LR = LogisticRegression(max_iter=200, random_state=SEED)
LR.fit(matriz_treino, treino.category)

LogisticRegression(max_iter=200, random_state=42)

In [27]:
print(LR.n_iter_)

[129]


In [28]:
LR.score(matriz_teste, teste.category)

0.7957880368546775

Utilizando os parâmetros default (solver `lbfgs`) o algoritimo executou bem mais rápido e com o mesmo score de solvers mais complexos, como o `saga`.

In [29]:
teste.category.unique()

array(['colunas', 'esporte', 'mercado', 'cotidiano', 'mundo', 'ilustrada'],
      dtype=object)

Como ele está se saindo para cada categoria?
Verificar o relatório das métricas com `classification_report`

In [30]:
label_prevista = LR.predict(matriz_teste)

In [31]:
CR = classification_report(teste.category, label_prevista)
print(CR)

              precision    recall  f1-score   support

     colunas       0.86      0.71      0.78      6103
   cotidiano       0.61      0.79      0.69      1698
     esporte       0.92      0.88      0.90      4663
   ilustrada       0.13      0.88      0.23       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.68      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



É semelhante, em alguns aspectos, aos dados para a matriz de confusão.

* **Precision (Precisão)**: Do que foi classificado como A, apenas x% é a classificação predita.
* **Recall (Revocação)**: De todos os registros que eram da categoria A, x% foram preditos corretamente como A.
* **F1-Score**: Média harmônica das métricas anteriores.

É notável que a categoria com maior erro é a **ilustrada**. A maior parte do que se classifica como ela foi corretamente classificado, mas boa parte das classificações foi errada. No entanto ela é a cateogoria com menor `support`, ou seja, com menor quantidade de itens (apenas 131) e, portanto, seu impacto não é tão grande na base. Inclusive seu score ruim pode ser justamente pela baixa quantidade de dados.

Será que estamos tendo _overfitting_? Vamos comparar o score com os dados de treino:

In [32]:
label_prevista = LR.predict(matriz_treino)
CR = classification_report(treino.category, label_prevista)
print(f'Score: {LR.score(matriz_treino, treino.category)}\n\n')
print(CR)

Score: 0.8166555555555556


              precision    recall  f1-score   support

     colunas       0.76      0.72      0.74     15000
   cotidiano       0.80      0.81      0.81     15000
     esporte       0.90      0.89      0.89     15000
   ilustrada       0.83      0.84      0.84     15000
     mercado       0.78      0.80      0.79     15000
       mundo       0.83      0.85      0.84     15000

    accuracy                           0.82     90000
   macro avg       0.82      0.82      0.82     90000
weighted avg       0.82      0.82      0.82     90000



O Score para treino é maior, mas não tão maior. No entanto é perceptível o desbalanceamento das categorias entre treino e teste.

### Validando os resultados com o `DummyClassifier`

In [33]:
DC = DummyClassifier(random_state=SEED, strategy='prior')
DC.fit(matriz_treino, treino.category)

label_prevista_dc = DC.predict(matriz_teste)
CR = classification_report(teste.category, label_prevista_dc, zero_division=0)
print(f'Test Score: {DC.score(matriz_teste, teste.category)}')
print(f'Train Score: {DC.score(matriz_treino, treino.category)}\n\n')
print(CR)

Test Score: 0.29751864671184125
Train Score: 0.16666666666666666


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



No melhor dos casos o `DummyClassifier` tem precisão de 30% ao chutar tudo como "colunas". Isso indica que o nosso classificador com acurácia de aproximadamente 80% é bom.

## Utilizando Skip-gram

In [34]:
modelo_sg = KeyedVectors.load_word2vec_format('dados/skip_s300.txt')

In [35]:
def combinacao_de_vetores_por_soma_sg(palavras_numeros):
    vetor_resultante = np.zeros(300)
    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo_sg.get_vector(pn)
        except KeyError:
            if(pn.isnumeric()):
                pn = "0" * len(pn)
            else:
                pn = "unknown"
            vetor_resultante += modelo_sg.get_vector(pn)
    return vetor_resultante

In [36]:
def matriz_vetores_sg(textos):
    x = len(textos)
    matriz = np.zeros((x, 300))
    for i in range(x):
        tokens = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma_sg(tokens)
    return matriz

In [37]:
matriz_treino_sg = matriz_vetores_sg(treino.title)
matriz_teste_sg = matriz_vetores_sg(teste.title)

In [38]:
LR = LogisticRegression(max_iter=300, random_state=SEED)
LR.fit(matriz_treino_sg, treino.category)

LogisticRegression(max_iter=300, random_state=42)

In [39]:
label_prevista = LR.predict(matriz_teste_sg)
CR = classification_report(teste.category, label_prevista, zero_division=0)
print(f'Test Score: {LR.score(matriz_teste_sg, teste.category)}')
print(f'Train Score: {LR.score(matriz_treino_sg, treino.category)}\n\n')
print(CR)

Test Score: 0.8068541900258372
Train Score: 0.8314333333333334


              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.91      0.26       131
     mercado       0.84      0.82      0.83      5867
       mundo       0.76      0.86      0.80      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513



Foi necessário aumentar o número de interações, mas foi obtido uma pequena melhora no score.

# Word2Vec: Treinamento de Word Embedding utilizando o Spacy

Lembrando que `Word2Vec` é uma rede neural.

In [40]:
nlp = spacy.load("pt_core_news_sm")
nlp

In [41]:
texto = "Rio de Janeiro é uma cidade violenta"
doc = nlp(texto)
print(doc)
print(type(doc))

Rio de Janeiro é uma cidade violenta
<class 'spacy.tokens.doc.Doc'>


In [42]:
print(f'Rio: {doc[0].is_stop}')
print(f'de: {doc[1].is_stop}')

Rio: False
de: True


Essa biblioteca já identifica automaticamente o que são as _stop words_.

In [43]:
titulos_treino = (t.lower() for t in treino.title)
titulos_teste = (t.lower() for t in teste.title)

In [44]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if(e_valido):
            tokens_validos.append(token.text)
    if(len(tokens_validos) > 2):
        return " ".join(tokens_validos)

In [45]:
trata_textos(doc)

'Rio Janeiro cidade violenta'

In [46]:
textos_tratados_treino = [trata_textos(doc) for doc in nlp.pipe(titulos_treino, batch_size=1000, n_process=-1)]
textos_tratados_teste = [trata_textos(doc) for doc in nlp.pipe(titulos_teste, batch_size=1000, n_process=-1)]

In [47]:
textos_tratados_treino

['polêmica marine le pen abomina negacionistas holocausto',
 'macron le pen turno frança revés siglas tradicionais',
 'apesar larga vitória legislativas macron terá desafios frente',
 'governo antecipa balanço alckmin anuncia queda homicídios sp',
 'queda maio atividade econômica sobe junho bc',
 'barcelona vence virada atlético madri bate bayern munique',
 'spartacus oferece duplo retrato batalhas perdidas',
 'sobe mortos atentado terrorista nice frança',
 'premiada sundance crystal moselle retrata sexismo mundo skate',
 'metroviários ferroviários ameaçam parar terça paulo',
 'anos angeli tirinhas diárias ilustrada',
 'mortes acidentes trânsito caem cidade sp',
 'daniel craig será stormtrooper star wars ator',
 'justiça sp libera construção moradias manancial billings',
 'prefiro urnas julguem lula achem batom cueca',
 'olivier anquier transformar cobertura centro restaurante',
 'mvp nba temporadas steve nash anuncia aposentadoria',
 'frança nega vinculo terrorista ataque soldados mes

In [48]:
df_treino = pd.DataFrame({"titulo": textos_tratados_treino})
df_teste = pd.DataFrame({"titulo": textos_tratados_teste})

In [49]:
df_treino = df_treino.dropna().drop_duplicates()
df_teste = df_teste.dropna().drop_duplicates()

In [50]:
df_treino.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


Documentação da `Word2Vec` disponível [aqui](https://radimrehurek.com/gensim/models/word2vec.html).

In [51]:
W2V = Word2Vec(sg = 0, window = 2, vector_size = 300, min_count = 5, alpha = 0.03, min_alpha = 0.007)
W2V

In [52]:
ll_tokens = [titulo.split(' ') for titulo in df_treino.titulo]
logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)
W2V = Word2Vec(sg = 0, window = 2, vector_size = 300, min_count = 5, alpha = 0.03, min_alpha = 0.007)
W2V.build_vocab(ll_tokens, progress_per=5000)

2021-10-16 10:59:19,307 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2021-10-16T10:59:19.307660', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-10-16 10:59:19,309 : - collecting all words and their counts
2021-10-16 10:59:19,311 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-16 10:59:19,331 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-10-16 10:59:19,350 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-10-16 10:59:19,376 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-10-16 10:59:19,397 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-10-16 10:59:19,420 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word

In [53]:
W2V.train(ll_tokens, total_examples = W2V.corpus_count, epochs = 30)

2021-10-16 10:59:20,324 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2021-10-16T10:59:20.324866', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
2021-10-16 10:59:21,273 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 10:59:21,275 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 10:59:21,292 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 10:59:21,293 : - EPOCH - 1 : training on 540242 raw words (485990 effective words) took 0.9s, 515674 effective words/s
2021-10-16 10:59:22,222 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 10:59:22,225 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 10:59:22,247 : - worker thre

2021-10-16 10:59:38,244 : - EPOCH - 19 : training on 540242 raw words (486057 effective words) took 1.0s, 469573 effective words/s
2021-10-16 10:59:39,240 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 10:59:39,261 : - EPOCH 20 - PROGRESS: at 98.16% examples, 476627 words/s, in_qsize 1, out_qsize 1
2021-10-16 10:59:39,262 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 10:59:39,272 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 10:59:39,273 : - EPOCH - 20 : training on 540242 raw words (486115 effective words) took 1.0s, 479438 effective words/s
2021-10-16 10:59:40,292 : - EPOCH 21 - PROGRESS: at 96.23% examples, 462796 words/s, in_qsize 3, out_qsize 0
2021-10-16 10:59:40,295 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 10:59:40,298 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 10:59:40,324 : - worker thread finished; awaiting finish of 0 more threads
2021

(14584231, 16207260)

In [54]:
W2V.wv.most_similar("google")

[('apple', 0.5593637228012085),
 ('facebook', 0.5288230776786804),
 ('amazon', 0.46059781312942505),
 ('uber', 0.4591052532196045),
 ('walmart', 0.4446199834346771),
 ('waze', 0.4441661536693573),
 ('canais', 0.42972439527511597),
 ('software', 0.4283818006515503),
 ('volkswagen', 0.4207298457622528),
 ('airbnb', 0.42021358013153076)]

In [55]:
W2V_sg = Word2Vec(sg = 1, window = 5, vector_size = 300, min_count = 5, alpha = 0.03, min_alpha = 0.007)
W2V_sg.build_vocab(ll_tokens, progress_per=5000)
W2V_sg.train(ll_tokens, total_examples = W2V_sg.corpus_count, epochs = 30)
W2V_sg.wv.most_similar("google")

2021-10-16 10:59:46,926 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2021-10-16T10:59:46.926556', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-10-16 10:59:46,927 : - collecting all words and their counts
2021-10-16 10:59:46,928 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-16 10:59:46,943 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-10-16 10:59:46,956 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-10-16 10:59:46,968 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-10-16 10:59:46,977 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-10-16 10:59:46,992 : - PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word

2021-10-16 11:00:03,181 : - EPOCH 8 - PROGRESS: at 55.60% examples, 256210 words/s, in_qsize 6, out_qsize 0
2021-10-16 11:00:03,886 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 11:00:03,924 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 11:00:03,931 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 11:00:03,932 : - EPOCH - 8 : training on 540242 raw words (486073 effective words) took 1.8s, 269501 effective words/s
2021-10-16 11:00:04,972 : - EPOCH 9 - PROGRESS: at 57.46% examples, 270980 words/s, in_qsize 5, out_qsize 0
2021-10-16 11:00:05,594 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 11:00:05,673 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 11:00:05,681 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 11:00:05,682 : - EPOCH - 9 : training on 540242 raw words (486084 effective words) took 1.7s, 279507 effective words/s
2021-10-

2021-10-16 11:00:31,183 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 11:00:31,185 : - EPOCH - 24 : training on 540242 raw words (486209 effective words) took 1.9s, 254956 effective words/s
2021-10-16 11:00:32,258 : - EPOCH 25 - PROGRESS: at 48.21% examples, 222021 words/s, in_qsize 5, out_qsize 0
2021-10-16 11:00:33,196 : - worker thread finished; awaiting finish of 2 more threads
2021-10-16 11:00:33,242 : - worker thread finished; awaiting finish of 1 more threads
2021-10-16 11:00:33,252 : - worker thread finished; awaiting finish of 0 more threads
2021-10-16 11:00:33,253 : - EPOCH - 25 : training on 540242 raw words (486072 effective words) took 2.0s, 237178 effective words/s
2021-10-16 11:00:34,272 : - EPOCH 26 - PROGRESS: at 42.66% examples, 206333 words/s, in_qsize 5, out_qsize 0
2021-10-16 11:00:35,288 : - EPOCH 26 - PROGRESS: at 90.67% examples, 218387 words/s, in_qsize 5, out_qsize 0
2021-10-16 11:00:35,425 : - worker thread finished; awaiting finish

[('reguladores', 0.4009733200073242),
 ('android', 0.3942333459854126),
 ('apple', 0.3780948221683502),
 ('waze', 0.3773370385169983),
 ('buffett', 0.3699413239955902),
 ('toshiba', 0.3659222424030304),
 ('concorda', 0.3552119731903076),
 ('xp', 0.3488658666610718),
 ('verizon', 0.34815293550491333),
 ('yahoo', 0.34557196497917175)]

Note que esse modelo foi capaz de identificar android, anunciantes, patentes e outros. Ou seja, ele traz uma visão mais abrangente do contexto que o anterior.

### Salvando o modelo

In [56]:
W2V.wv.save_word2vec_format('dados/w2v_cbow.txt', binary=False)
W2V_sg.wv.save_word2vec_format('dados/w2v_sg.txt', binary=False) 

2021-10-16 11:00:47,754 : - storing 12924x300 projection weights into dados/w2v_cbow.txt
2021-10-16 11:00:52,406 : - storing 12924x300 projection weights into dados/w2v_sg.txt


## Reescrevendo as funções para classificação

In [57]:
nlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner', 'tagger', 'textcat'])

In [62]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('dados/w2v_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('dados/w2v_sg.txt')

2021-10-16 11:03:38,732 : - loading projection weights from dados/w2v_cbow.txt
2021-10-16 11:03:40,751 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from dados/w2v_cbow.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-10-16T11:03:40.751612', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}
2021-10-16 11:03:40,752 : - loading projection weights from dados/w2v_sg.txt
2021-10-16 11:03:42,592 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from dados/w2v_sg.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-10-16T11:03:42.592311', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}


In [58]:
def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if(e_valido):
            tokens_validos.append(token.text.lower())
    return tokens_validos

In [60]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros((1,300))
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass
    return vetor_resultante

In [63]:
palavras = tokenizador('Rio de Janeiro 12312312 *** @# é uma cidade violenta')
print(palavras)
valor_palavras = combinacao_de_vetores_por_soma(palavras, w2v_modelo_sg)
print(valor_palavras)
valor_palavras = combinacao_de_vetores_por_soma(palavras, w2v_modelo_cbow)
print(valor_palavras)

['rio', 'janeiro', 'cidade', 'violenta']
[[ 2.15897411e-01  9.39826518e-01 -8.71154238e-02  3.01420801e-01
   3.43035802e-01 -2.03541011e-01  7.03541040e-01  1.21816064e+00
  -9.34463330e-02 -5.59411131e-01  4.82271016e-01  2.94023968e-01
  -1.49650574e+00 -1.34549583e-01  3.43235219e-01 -5.14608982e-01
   9.68449950e-01 -8.01745353e-01 -9.60245728e-03 -1.76537402e-01
  -1.40559870e+00 -2.93939462e-01  2.06733586e-01  3.29322964e-02
  -5.09615824e-01 -6.36634706e-01 -5.15210003e-01 -4.60097924e-01
  -8.42132008e-01 -5.33972360e-01 -8.16065483e-01  1.09470444e-01
   3.70571649e-01  1.08355076e-01  7.46704022e-01  8.18956237e-01
  -1.23409495e-01 -1.44539481e+00 -1.30575366e-01 -4.94200635e-01
  -1.14176811e+00  2.20861062e-01  3.88324089e-01 -4.48847301e-01
   2.99152762e-01 -1.10024601e-01  5.13883293e-01  6.31684631e-01
   1.47480025e+00  1.07255043e+00  8.68336096e-01 -1.08466700e+00
  -8.70950490e-01 -1.03536734e+00  4.50479975e-01  4.65657886e-01
   4.25870970e-01 -6.24014296e-01 -

In [64]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x, y))
    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
    return matriz

In [65]:
matriz_vetores_treino_cbow = matriz_vetores(treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [76]:
matriz_vetores_treino_sg = matriz_vetores(treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(90000, 300)
(20513, 300)


In [70]:
def classificador(X_treino, y_treino, X_teste, y_teste):
    LR = LogisticRegression(max_iter=800, random_state=SEED)
    LR.fit(X_treino, y_treino)
    label_prevista = LR.predict(X_teste)
    CR = classification_report(y_teste, label_prevista, zero_division=0)
    print(f'Test Score: {LR.score(X_teste, y_teste)}')
    print(f'Train Score: {LR.score(X_treino, y_treino)}\n\n')
    print(CR)
    return (LR, CR)

In [74]:
LR_cbow, CR_cbow = classificador(matriz_vetores_treino_cbow, treino.category, matriz_vetores_teste_cbow, teste.category)

Test Score: 0.7845268853897528
Train Score: 0.8107


              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.81      0.78      0.80     20513



In [77]:
LR_sg, CR_sg = classificador(matriz_vetores_treino_sg, treino.category, matriz_vetores_teste_sg, teste.category)

Test Score: 0.7952030419733828
Train Score: 0.8202444444444444


              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.15      0.89      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



## Salvando o modelo

In [82]:
with open('dados/lr_cbow.pkl', 'wb') as f:
    pickle.dump(LR_cbow, f)

with open('dados/lr_sg.pkl', 'wb') as f:
    pickle.dump(LR_sg, f)